In [1]:
import qiskit
import numpy as np
import math
import random
from qiskit import providers,execute,QuantumRegister,ClassicalRegister,QuantumCircuit,transpile,Aer,assemble,IBMQ
from qiskit.tools import job_monitor
from qiskit.providers.aer import AerSimulator
from qiskit.visualization import plot_histogram
from numpy.random import randint,random_integers
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import thermal_relaxation_error
aer_sim = Aer.get_backend('aer_simulator')


In [2]:
def get_noise(p_bit,p_phase,p_gate1):
    bit_flip = pauli_error([('X',p_bit), ('I', 1 - p_bit)])
    phase_flip = pauli_error([('Z', p_phase), ('I', 1 - p_phase)])
    error_gate1 = depolarizing_error(p_gate1, 1)
    t_error = thermal_relaxation_error(0.0125, 0.0025, 0.01)
   
    noise_model = NoiseModel()
    # Measurement error had been applied 
    #noise_model.add_all_qubit_quantum_error(bit_flip, "measure") 
    # Depolarizing channel had been applied 
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x","h"]) 
    # Thermal relaxation error had been applied 
    #noise_model.add_all_qubit_quantum_error(t_error,["x","h"])    
    return noise_model

    

In [3]:
def encode_message(bits,bases):
    message=[]
    for i in range(n):
        qc=QuantumCircuit(1,1)
        if(bases[i]==1):#z basis
            if(bits[i]==0):
                pass#creating |0> state 
            else:
                qc.x(0)#creating |1> state 
        else:#x basis
            if(bits[i]==0):
                qc.h(0)#creating |+> state 
            else:
                qc.x(0)
                qc.h(0)#creating |-> state
        message.append(qc)
    return message
  
def measure_message(message,bases): 
    measured=[]
    for j in range(n):
        if(bases[j]==1):# z bases
            message[j].measure(0,0)
        else:# x bases
            message[j].h(0)
            message[j].measure(0,0)
        simul=Aer.get_backend('aer_simulator')
        qobj=assemble(message[j],shots=8192,memory=True)
        result=simul.run(qobj,noise_model=noise_model).result()
        measurement=int(result.get_memory()[0])
        measured.append(measurement)
    return measured

def key_selection(s_bases,r_bases,bits):
    good_bits=[]  
    for i in range(n):
        if(s_bases[i]==r_bases[i]):
            good_bits.append(bits[i])
    return good_bits

def sampling(bits,string):
    sample=[]
    for i in string:
        i=np.mod(i,len(bits))#ensuring that the sample string doesnot exceed the key
        sample.append(bits.pop(i))
    return sample

'''
Function to calcuate the enhanced key in absence of eve
'''
def enhanced_key():
    enhanced_key=[]
    enhanced_key_r=[]
    for i in range(63):
        sender_bits=np.random.randint(2,size=n)
        sender_basis=randint(2,size=n)
        reciever_basis=randint(2,size=n)
        data=encode_message(sender_bits,sender_basis)
        key=measure_message(data,reciever_basis)
        s_key=key_selection(sender_basis,reciever_basis,sender_bits)
        for j in range(len(s_key)):
            enhanced_key.append(s_key[j])
        r_key=key_selection(sender_basis,reciever_basis,key)
        for j in range(len(r_key)):
            enhanced_key_r.append(r_key[j])
        
    return enhanced_key,enhanced_key_r
'''
Function to calcuate the enhanced key in presence of eve
'''
def enhanced_key_eve():
    enhanced_key=[]
    enhanced_key_r=[]
    for i in range(63):
        sender_bits=np.random.randint(2,size=n)
        sender_basis=randint(2,size=n)
        attacker_basis=randint(2,size=n)
        reciever_basis=randint(2,size=n)
        data=encode_message(sender_bits,sender_basis)
        intercepted_message = measure_message(data,attacker_basis)
        key=measure_message(data,reciever_basis)
        s_key=key_selection(sender_basis,reciever_basis,sender_bits)
        for j in range(len(s_key)):
            enhanced_key.append(s_key[j])
        r_key=key_selection(sender_basis,reciever_basis,key)
        for j in range(len(r_key)):
            enhanced_key_r.append(r_key[j])
        
    return enhanced_key,enhanced_key_r

In [4]:
#Without Eve
from qiskit.circuit import quantumcircuit
n=32
noise_model = get_noise(0.1,0.1,0.1)#Omit the line to get noisless enviorment
s_key,r_key=enhanced_key()
print("The length of the shifted key is:")
print(len(s_key))

'''
Keeping the sample size 25% of the shifted key length
'''
s=math.ceil(len(s_key) * 0.25)
print(" sample .... ", s)
sample_size=randint(0,s)
'''
Sampling is doen with 25% of shifted key length
'''
print("length of sifted key is ",len(s_key) )
print("length of sample taken is ",sample_size)
sample_string = randint(n, size=sample_size)
sender_sample=sampling(s_key,sample_string)
reciever_sample=sampling(r_key,sample_string)
'''
Code to calcuate the error rate e
'''
mismatch = 0
for i in range(sample_size):
    if(sender_sample[i]!=reciever_sample[i]):
        mismatch = mismatch+1
print("Length of the sender sample is")
print(len(sender_sample))
print("Length of the receiver sample is")
print(len(reciever_sample))


if sender_sample==reciever_sample:
    print("Cryptographic_key_is length is")
    print(len(s_key))
else:
    print("Error")
    error_rate=mismatch/sample_size
    print("Error_rate=",error_rate)
        
    

Traceback (most recent call last):
  Cell In[4], line 5
    s_key,r_key=enhanced_key()
  Cell In[3], line 59 in enhanced_key
    key=measure_message(data,reciever_basis)
  Cell In[3], line 29 in measure_message
    result=simul.run(qobj,noise_model=noise_model).result()
NameError: name 'noise_model' is not defined

Use %tb to get the full traceback.


In [ ]:
#In presence of Eve
from qiskit.circuit import quantumcircuit
#np.random.seed(seed=0)
n=32
noise_model = get_noise(0.05,0.05,0.05)
s_key,r_key=enhanced_key_eve()
print("The length of the shifted key is:")
print(len(s_key))


s=math.ceil(len(s_key) * 0.25)
print(" sample .... ", s)
sample_size=randint(0,s)

print("length of sifted key is ",len(s_key) )
print("length of sample taken is ",sample_size)
sample_string = randint(n, size=sample_size)
sender_sample=sampling(s_key,sample_string)
reciever_sample=sampling(r_key,sample_string)

mismatch = 0
for i in range(sample_size):
    if(sender_sample[i]!=reciever_sample[i]):
        mismatch = mismatch+1
print("Length of the sender sample is")
print(len(sender_sample))
print("Length of the receiver sample is")
print(len(reciever_sample))


if sender_sample==reciever_sample:
    print("Cryptographic_key_is length is")
    print(len(s_key))
else:
    print("Error")
    error_rate=mismatch/sample_size
    print("Error_rate=",error_rate)
        
    

In [6]:
def skr(shift,e):
    h=e*(math.log(e,2))+(1-e)*(math.log((1-e),2))
    tau=-(math.log((0.5+2*e-2*e*e),2))
    if(e==0):
        f=0
    if(e==0.06493506493506493):
        f=1.16
    if(e==0.075):
        f=1.22
    if(e==0.15):
        f=1.35
    kr=shift*(tau+(f*h))
    return kr

In [8]:
print(skr(1010,0.075))

179.59479125280467
